In [1]:
import telegram 
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import CommandHandler, CallbackQueryHandler, CallbackContext, ApplicationBuilder
import pandas as pd
from pandas.plotting import table 
import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
import requests

from source.bot_source import LabellingBot
import nest_asyncio

from tqdm import tqdm

In [88]:
nllb, mbart, pairs, _, tartu = pd.read_csv('data\\trans_10k_all.csv').iloc[0]

In [89]:
pairs = pairs[1:-1].split('), ')
pairs = [tuple(p[1:].split(', \'')) for p in pairs]
pairs = [(f[1:-1], s[:-1]) for f, s in pairs]

In [90]:
pairs

[('Nevvostoliitos', 'кон — кон (учреждение)'),
 ('kehitettih', 'формировать (придавать форму)'),
 ('omaluaduine,', 'эксцентричный'),
 ('pehmei', 'палатальный'),
 ('taba', 'характер (нрав)'),
 ('kuvata', 'списать (воспроизвести по какому-л. оригиналу)'),
 ('liikehty,', 'укачивание'),
 ('kudamas', 'пресн|ый (о пище)'),
 ('tuli', 'электричеств|о'),
 ('muan', 'земн|ой'),
 ('animacien', 'плюгавый'),
 ('tunnusmerki.', "эмблема'")]

In [24]:
nllb

'В бракосочетании разработали своеобразный, мягкий способ изображения движения, который стал признаком земной анимиции.'

In [4]:
lab = pd.read_csv('data/labeled.csv').drop_duplicates(ignore_index=True)

In [5]:
# %pip install python-telegram-bot[rate-limiter]

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import requests

# headers = {
#     'accept': 'application/json',
#     'Content-Type': 'application/json',
# }
# res = list()
# text = 'Sen jälles häi muutti Yhtysvaldoih da luadi sie animaciesarjua "Newlyweds", kudamua imitiiruittih äijän'
# for text in tqdm(lab['kar'].values):
#     json_data = {
#         'text': text,
#         'src': 'olo',
#         'tgt': 'rus',
#         'domain': 'general',
#         'application': 'Documentation UI',
#     }

#     response = requests.post('https://api.tartunlp.ai/translation/v2', headers=headers, json=json_data)
#     res.append(response.json()['result'])

: 

# Бот в отдельном файле

In [7]:
nest_asyncio.apply()

bot = LabellingBot()
bot.run()

2025-05-24 02:07:47,818 - telegram.ext.Application - INFO - Application started


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

2025-05-24 02:09:52,432 - telegram.ext.Application - INFO - Application is stopping. This might take a moment.
2025-05-24 02:09:52,434 - telegram.ext.Application - INFO - Application.stop() complete


RuntimeError: Cannot close a running event loop

# Отладка

In [4]:
line = 'Aazii'
line.split('.')

['Aazii']

In [5]:
line[-1] != '.'

True

In [16]:
ab = [1]
ab[1:]

[]

In [20]:
pairs = [('Časounu-sana', 'часовня'),
  ('tulou', 'прийтись (безл.)'),
  ("ven'an", 'русско-'),
  ('sanas', 'лексика'),
  ('"časovnja"', 'часовня'),
  ('(),', 'воодушевить(с'),
  ('kuduas', 'где'),
  ('omas', 'собственн|ый'),
  ('puoles', 'по (при указании на область деятельности)'),
  ('on', 'есть (безл. в знач. сказ.)'),
  ('"čas"-kandu', 'искушени|е (испытание)')]

In [37]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
a = np.ones((1, 10))
b = np.array([1])
cosine_similarity(a, b.reshape(1, -1)) # 1x1

In [44]:
b.shape

(1,)

In [22]:
msg=f"Dictionary:\n{'\n'.join([f'{k} -> {v}' for k, v in pairs])}"
print(msg)

Dictionary:
Časounu-sana -> часовня
tulou -> прийтись (безл.)
ven'an -> русско-
sanas -> лексика
"časovnja" -> часовня
(), -> воодушевить(с
kuduas -> где
omas -> собственн|ый
puoles -> по (при указании на область деятельности)
on -> есть (безл. в знач. сказ.)
"čas"-kandu -> искушени|е (испытание)


In [54]:
text = 'asdfasdfas'
text_split = text.split() if len(text.split()) > 1 else [text]
print(text_split)

['asdfasdfas']


In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from tqdm import tqdm

def split_sentences(text, language='fi'):
    sentences = split_text_into_sentences(text=text, language=language) #.replace('\r\n', '')
    return [s.strip() for s in sentences if s.strip()]

file_path = os.path.join('data', 'corpus_to_label.txt')
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

print(split_sentences(lines[3], 'fi'))
new_lines = list()
for line in tqdm(lines):
    new_art = split_sentences(line, 'fi')
    new_lines.extend(new_art)
with open('new_lines.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(new_lines))

['Enzimäi ”afri”-sanal muinazen Karfagenan eläjät kučuttih ristikanzoi, kuduat elettih linnan lähäl.',
 'Tämä nimi on sivottu ”afar”-sanah, kudai merkiččöy finikakse pölyy.',
 'Konzu Riman eläjät vallattih Karfagenan, hyö nimitettih sen Afriekakse.',
 'Myöhembäh Afriekakse ruvettih kuččuo tämän manderen kaikkii alovehii, a sit iččiegi mannerdu.']

In [ ]:
import logging
from typing import Dict

from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram.ext import (
    Application,
    CommandHandler,
    ContextTypes,
    ConversationHandler,
    MessageHandler,
    PicklePersistence,
    filters,
)

# Enable logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
# set higher logging level for httpx to avoid all GET and POST requests being logged
logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

reply_keyboard = [
    ["Age", "Favourite colour"],
    ["Number of siblings", "Something else..."],
    ["Done"],
]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)


def facts_to_str(user_data: Dict[str, str]) -> str:
    """Helper function for formatting the gathered user info."""
    facts = [f"{key} - {value}" for key, value in user_data.items()]
    return "\n".join(facts).join(["\n", "\n"])


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Start the conversation, display any stored data and ask user for input."""
    reply_text = "Hi! My name is Doctor Botter."
    if context.user_data:
        reply_text += (
            f" You already told me your {', '.join(context.user_data.keys())}. Why don't you "
            "tell me something more about yourself? Or change anything I already know."
        )
    else:
        reply_text += (
            " I will hold a more complex conversation with you. Why don't you tell me "
            "something about yourself?"
        )
    await update.message.reply_text(reply_text, reply_markup=markup)

    return CHOOSING


async def regular_choice(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text.lower()
    context.user_data["choice"] = text
    if context.user_data.get(text):
        reply_text = (
            f"Your {text}? I already know the following about that: {context.user_data[text]}"
        )
    else:
        reply_text = f"Your {text}? Yes, I would love to hear about that!"
    await update.message.reply_text(reply_text)

    return TYPING_REPLY


async def custom_choice(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Ask the user for a description of a custom category."""
    await update.message.reply_text(
        'Alright, please send me the category first, for example "Most impressive skill"'
    )

    return TYPING_CHOICE


async def received_information(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Store info provided by user and ask for the next category."""
    text = update.message.text
    category = context.user_data["choice"]
    context.user_data[category] = text.lower()
    del context.user_data["choice"]

    await update.message.reply_text(
        "Neat! Just so you know, this is what you already told me:"
        f"{facts_to_str(context.user_data)}"
        "You can tell me more, or change your opinion on something.",
        reply_markup=markup,
    )

    return CHOOSING


async def show_data(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Display the gathered info."""
    await update.message.reply_text(
        f"This is what you already told me: {facts_to_str(context.user_data)}"
    )


async def done(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Display the gathered info and end the conversation."""
    if "choice" in context.user_data:
        del context.user_data["choice"]

    await update.message.reply_text(
        f"I learned these facts about you: {facts_to_str(context.user_data)}Until next time!",
        reply_markup=ReplyKeyboardRemove(),
    )
    return ConversationHandler.END


def main() -> None:
    """Run the bot."""
    # Create the Application and pass it your bot's token.
    persistence = PicklePersistence(filepath="conversationbot")
    token = ''
    application = Application.builder().token(token).persistence(persistence).build()

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("start", start)],
        states={
            CHOOSING: [
                MessageHandler(
                    filters.Regex("^(Age|Favourite colour|Number of siblings)$"), regular_choice
                ),
                MessageHandler(filters.Regex("^Something else...$"), custom_choice),
            ],
            TYPING_CHOICE: [
                MessageHandler(
                    filters.TEXT & ~(filters.COMMAND | filters.Regex("^Done$")), regular_choice
                )
            ],
            TYPING_REPLY: [
                MessageHandler(
                    filters.TEXT & ~(filters.COMMAND | filters.Regex("^Done$")),
                    received_information,
                )
            ],
        },
        fallbacks=[MessageHandler(filters.Regex("^Done$"), done)],
        name="my_conversation",
        persistent=True,
    )

    application.add_handler(conv_handler)

    show_data_handler = CommandHandler("show_data", show_data)
    application.add_handler(show_data_handler)

    # Run the bot until the user presses Ctrl-C
    application.run_polling(allowed_updates=Update.ALL_TYPES)

In [ ]:
nest_asyncio.apply()
main()

2025-04-11 18:35:35,800 - telegram.ext.Application - INFO - Application started
2025-04-11 18:37:39,684 - telegram.ext.Application - INFO - Application is stopping. This might take a moment.
2025-04-11 18:37:39,685 - telegram.ext.Application - INFO - Application.stop() complete


RuntimeError: Cannot close a running event loop